In [4]:
import numpy as np
import pylab as plt
import matplotlib
import pyLikelihood
import os
import glob 
from fermipy.gtanalysis import GTAnalysis
from fermipy.plotting import ROIPlotter, SEDPlotter
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import brentq
from scipy.optimize import minimize_scalar
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy import interpolate
from scipy.interpolate import UnivariateSpline
from fermipy import utils

## 1- Importando os dados de maneira inteligente

In [5]:
# Vou utilizar dados de P8R2 para 6 e 14 anos
list_lnl_14years_P8R2 = glob.glob('eflux_and_loglikes_files/P8R2_14Years_files/*_Dloglikes_14years_P8R2.txt')
list_lnl_6years_P8R2 = glob.glob('eflux_and_loglikes_files/P8R2_6years_Files/*_Dloglikes_6years_P8R2.txt')

#-----------------------------------------#
list_eflux_14years_P8R2 = glob.glob('eflux_and_loglikes_files/P8R2_14Years_files/*_eflux_14years_P8R2.txt')
list_eflux_6years_P8R2  = glob.glob('eflux_and_loglikes_files/P8R2_6years_Files/*_eflux_6years_P8R2.txt*')

#criando lista de Jfactor

Jfactor_array = np.array([18.2,17.6,17.9,19.0,18.8,17.8,16.9,18.0,16.3,18.5,19.4,17.5,19.4,18.9,18.9])

# criando Lista de likefiles
list_likefile = glob.glob('likes_files/like_*')

# Demais dados
specfile = 'spectrum_bbbar_100GeV.txt' # Esse txt conseguir em um dos primeiros artigos do fermi
crossesctions = np.loadtxt('limits_bb.txt',unpack=True)

### 1.1 Definindo variaveis para funcionalidade do código

In [47]:
data = np.loadtxt(list_likefile[0], unpack=True)
emins, emaxs = np.unique(data[0]),np.unique(data[1])
ebin = np.sqrt(emins*emaxs)
efluxes = data[2].reshape(len(emins),-1)
logLikes = data[3].reshape(len(emins),-1)
Spectrum =np.loadtxt(specfile,unpack=True)
mass = crossesctions[0]
sigmav0 = 1e-25
delta =2.71

lista_de_nomes = ['Bootes','Canes_Venatici II','Carina','Coma_Berenices','Draco',
                  'Fornax','Hercules','Leo_II','Leo_IV','Sculptor','Segue_I','Sextans',
                  'Ursa_Major_II','Ursa Minor','Willma_1']  

lista_de_nomes_old = ['Bootes_I','Canes','Carina', 'Coma','Draco','Fornax','Hercules','Leo_II','Leo_IV','Sculptor','Segue_1',
                     'Sextans','Ursa_Major_II','Ursa_Minor','Wilma']
# Esse norm_scan é meio artistico de conseguir reproduzi-lo
norm_scan = np.loadtxt('norm_scan.txt')

# Estrutura física do códido

In [14]:
def eflux(spectrum, emin=1e2, emax=1e5, quiet=False):
    """ Integrate a generic spectrum, multiplied by E, to get the energy flux.
    """
    espectrum = lambda e: spectrum(e)*e
    tol = min(espectrum(emin),espectrum(emax))*1e-10
    try:
        return quad(espectrum,emin,emax,epsabs=tol,full_output=True)[0]
    except (Exception, msg):
        print('Numerical error "%s" when calculating integral flux.' % msg)
        return np.nan
# Keep numpy from complaining about dN/dE = 0...

In [ ]:
for i in range(len(Jfactor_array)):
    efluxes= np.loadtxt(list_eflux_6years_P8R2[i])
    dlog= np.loadtxt(list_lnl_6years_P8R2[i])
    lnl_fermi_scan = np.zeros((21,7000))
    jfactor_fermi   = Jfactor_array[i]
    j0_fermi      = jfactor_fermi
    k=0
    for m in mass:
        dmf = pyLikelihood.DMFitFunction()
        dmf.readFunction(os.path.expandvars('$FERMIPY_ROOT/data/gammamc_dif.dat'))
        dmf.setParam('norm',10**Jfactor_array[i])
        dmf.setParam('sigmav',1E-25)
        dmf.setParam('mass',m)
        dmf.setParam('bratio',1.0)
        dmf.setParam('channel0',4)
    
        energy = Spectrum[0] 
        dnde = np.zeros(energy.shape)
        for j in range(len(energy)):
            dnde[j] = dmf(pyLikelihood.dArg(energy[j]))
        
        log_energy = np.log10(energy)# Transforma energia para log
        log_dnde = np.log10(dnde)# Transforma Espectro para log
        log_interp = interp1d(log_energy,log_dnde) # Cria função de Interpolação entre energia e espectro
        spectrum = lambda e: np.nan_to_num(10**( log_interp(np.log10(e)) ))# Retorna um Valor de dn/de para um valor de energia
    
        #print(eflux.shape,dlog.shape)
        pred = np.array([eflux(spectrum,e1,e2) for e1,e2 in zip(emins,emaxs)])  # Retorna o Eflux esperado por Bin
        likes = [ interp1d(f,l-l.max(), fill_value='extrapolate') for f,l in zip(efluxes,dlog)] 
        like = lambda c: sum([lnlfn(c*p) for lnlfn,p in zip(likes,pred)])
    
        norms = norm_scan[k]
    
        lnl_fermi = np.array([like(n) for n in norms])
        lnl_fermi_scan[k] =lnl_fermi
        lnl_fermi_normalizada = lnl_fermi -lnl_fermi.max()
    
        print(lnl_fermi_normalizada.shape, m)
        sigmav = j0_fermi/jfactor_fermi * sigmav0 * norms
        #crossection_scan_P8R3[k] =sigmav
        mle = np.argmax(lnl_fermi_normalizada)
        lnlfn = interp1d(lnl_fermi_normalizada[mle:],sigmav[mle:])
        delta = 2.71/2
        limit = lnlfn(-delta)
        #crossection_mass_P8R3[k] = float(limit)
        k= k+1
        
    np.savetxt('New_cross_and_loglikes_files/Upper_limits_6years_P8R2/lnl_fermi_scan_'+lista_de_nomes[i],lnl_fermi_scan)


In [ ]:
fig =plt.figure(figsize=(12,6))
#plt.plot(mass,Crossection_P8R2,label= 'six_years_P8R2')
#plt.plot(mass,Crossection_P8R3, label = 'six_years_P8R3')
plt.plot(mass,crossection_mass_P8R2, label = '14 years P8R2')
plt.plot(mass,crossection_mass_P8R3, label = '6 anos ')
#plt.plot(mass,crossesctions[36], label = 'Arxiv :1611 - Fermi 2015')
#plt.plot(mass,crossection_mass_Fermi, label ='Arxiv :1611 - Fermi 2015 without J factor')
plt.xlabel('mass (Gev)')
plt.ylabel('Cross Section upper limit at 95%')
plt.title('Crossections vs Mass')
plt.xscale('log')
plt.yscale('log')
plt.text(2.5e3,1e-24,'Wilma', style='oblique',fontsize='15')
plt.legend()